In [1]:
import awkward as ak
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema

fname = "root://eospublic.cern.ch//eos/opendata/cms/Run2016H/DoubleMuon/NANOAOD/UL2016_MiniAODv2_NanoAODv9-v1/2510000/127C2975-1B1C-A046-AABF-62B77E757A86.root"
#fname = "root://localhost//disk03/opendata_cms/DoubleMuon/Run2016H-UL2016_MiniAODv2_NanoAODv9-v1/NANOAOD/127C2975-1B1C-A046-AABF-62B77E757A86.root"
events = NanoEventsFactory.from_root(
    {fname: "Events"},
    schemaclass=NanoAODSchema,
    metadata={"dataset": "DoubleMuon"},
).events()

print("Number of events = ", ak.num(events.Muon, axis=0).compute())

/usr/local/venv/lib/python3.10/site-packages/coffea/nanoevents/methods/candidate.py:11: FutureWarning: In version 2024.7.0 (target date: 2024-06-30 11:59:59-05:00), this will be an error.
To raise these warnings as errors (and get stack traces to find out where they're called), run
    import warnings
    warnings.filterwarnings("error", module="coffea.*")
after the first `import coffea` or use `@pytest.mark.filterwarnings("error:::coffea.*")` in pytest.
Issue: coffea.nanoevents.methods.vector will be removed and replaced with scikit-hep vector. Nanoevents schemas internal to coffea will be migrated. Otherwise please consider using that package!.
  from coffea.nanoevents.methods import vector


Number of events =  2147195


In [2]:
print("Number of muons = ", ak.count(events.Muon.mass, axis=None).compute())

Number of muons =  4806013


In [3]:
selected_muons = events.Muon[(events.Muon.isGlobal == 1)]
print("Number of selected muons after isGlobal==1: ", ak.count(selected_muons.mass, axis=None).compute())

Number of selected muons after isGlobal==1:  4110800


In [4]:
selected_muons = selected_muons[(selected_muons.nStations > 0)]
print("Number of selected muons after nStations>0: ", ak.count(selected_muons.mass, axis=None).compute())

Number of selected muons after nStations>0:  3936437


In [5]:
selected_muons = selected_muons[(selected_muons.nTrackerLayers > 5)]
print("Number of selected muons after nTrackerLayers>5: ", ak.count(selected_muons.mass, axis=None).compute())

Number of selected muons after nTrackerLayers>5:  3908889


In [6]:
selected_muons = selected_muons[(selected_muons.dxy < 0.3) & (selected_muons.dz < 20.0)]
print("Number of selected muons after PV impact param sel: ", ak.count(selected_muons.mass, axis=None).compute())       

Number of selected muons after PV impact param sel:  3892547


In [7]:
selected_muons = selected_muons[(selected_muons.pt > 3.0) & (abs(selected_muons.eta) < 2.4)]
print("Number of selected muons after pt,eta sel: ", ak.count(selected_muons.mass, axis=None).compute())

Number of selected muons after pt,eta sel:  3883500


In [8]:
selected_muons = selected_muons[(selected_muons.pfRelIso03_chg < 0.5)]
print("Number of selected muons after ptcone30 isolation: ", ak.count(selected_muons.mass, axis=None).compute())

Number of selected muons after ptcone30 isolation:  2681124


In [9]:
selected_muons_c = selected_muons.compute()

OSError: File did not vector_read properly: [ERROR] Operation expired

In [ ]:
def find_4lep(events_leptons, builder):
    """Search for valid 4-lepton combinations from an array of events * leptons {charge, ...}

    A valid candidate has two pairs of leptons that each have balanced charge
    Outputs an array of events * candidates {indices 0..3} corresponding to all valid
    permutations of all valid combinations of unique leptons in each event
    (omitting permutations of the pairs)
    """
    for leptons in events_leptons:
        builder.begin_list()
        nlep = len(leptons)
        for i0 in range(nlep):
            for i1 in range(i0 + 1, nlep):
                if leptons[i0].charge + leptons[i1].charge != 0:
                    continue
                for i2 in range(nlep):
                    for i3 in range(i2 + 1, nlep):
                        if len({i0, i1, i2, i3}) < 4:
                            continue
                        if leptons[i2].charge + leptons[i3].charge != 0:
                            continue
                        builder.begin_tuple(4)
                        builder.index(0).integer(i0)
                        builder.index(1).integer(i1)
                        builder.index(2).integer(i2)
                        builder.index(3).integer(i3)
                        builder.end_tuple()
        builder.end_list()

    return builder


In [ ]:
fourmuon = find_4lep(selected_muons_c, ak.ArrayBuilder()).snapshot()
if ak.all(ak.num(fourmuon) == 0):
  print("No four muons at all!")

fourmuon = [muons[fourmuon[idx]] for idx in "0123"]
fourmuon = ak.zip({
  "z1": ak.zip({
    "lep1": fourmuon[0],
    "lep2": fourmuon[1],
    "p4": fourmuon[0] + fourmuon[1],
  }),
  "z2": ak.zip({
    "lep1": fourmuon[2],
    "lep2": fourmuon[3],
    "p4": fourmuon[2] + fourmuon[3],
  }),
})

print(fourmuon)